In [6]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 pydrive -q
!pip install --upgrade pydrive google-colab google-auth -q

In [10]:
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import files

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = None
drive = GoogleDrive(gauth)

In [13]:
# Step 1: Upload the client_secrets.json file
uploaded = files.upload()

# Step 2: Move the uploaded file to the appropriate location
import shutil

for filename in uploaded.keys():
    shutil.move(filename, '/content/client_secrets.json')

# Step 3: Verify the file is in the correct location
!ls /content

Saving client_secret_427587412711-gf7tam713mg1deejjl4mr111u1vphsem.apps.googleusercontent.com.json to client_secret_427587412711-gf7tam713mg1deejjl4mr111u1vphsem.apps.googleusercontent.com.json
client_secrets.json  sample_data


In [14]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in file_list:
    if file['title'] == 'metal-smithy-387702-4be2e90f2441.json':
        credentials_file_id = file['id']
        break

InvalidConfigError: ignored